# Automated Rail Inclusion Application<br>
By Madeline Stelter<br>
mstelter001@regis.edu

Certain file pathways in this code have been replaced with dummy paths, and the files are not included in the Github repository. As such, this code will not run. This is to protect the security of the organization this code was developed for.

In [1]:
import numpy as np
import pandas as pd
# Libraries to use in the code

In [2]:
'''
Author: Bryce Lakamp, Senior Process Engineer, Evraz
Date: January 2023
bryce.lakamp@evrazna.com
'''
def rm_l2_mirror_conn_str():
        pw = open(r"(pathway to file)",'r').read()
        ht = open(r"(pathway to file)",'r').read()
        conn_str_parameters={

            'driver': 'SQL Server'.replace(' ', '+'),
            'user': 'dwreader',
            'pass': pw,
            'host': ht,
            'db_name': 'Prod_Tech_DW'

        return "mssql+pyodbc://{user}:{pass}@{host}/{db_name}?driver={driver}".format(**conn_str_parameters)

# Defined custom connection function to reach database. Developed by Bryce Lykamp. 
# Password (pw) and Server Host (ht) files are not included in the Github.

In [3]:
def st_l2_mirror_conn_str():

        pw1 = open(r"(pathway to file)",'r').read()
        ht1 = open(r"(pathway to file)",'r').read()
        
        conn_str_parameters={

            'driver': 'SQL Server'.replace(' ', '+'),
            'user': 'SMMirrorRead',
            'pass': pw1,
            'host': ht1,
            'db_name': 'RMSM_L2_Mirror'
        }


        return "mssql+pyodbc://{user}:{pass}@{host}/{db_name}?driver={driver}".format(**conn_str_parameters)

# Defined custom connection function to reach 2nd database using previous code as a guide.
# Password (pw) and Server Host (ht) files are not included in the Github.

In [4]:
'''
Author: Bryce Lakamp, Senior Process Engineer, Evraz
Date: January 2023
bryce.lakamp@evrazna.com
'''
if __name__ == "__main__":
    from IPython.display import display_html
    con = rm_l2_mirror_conn_str()
    sql = '''
    SELECT
        CONCAT(
            SEQ.HEAT
            , SEQ.RMSEQ
            , SEQ.RMLETTER
        ) as RAIL
        , HEAT
        , CONCAT( SEQ.RMSEQ, SEQ.RMLETTER) AS RAIL_ID
        , CASTID as CAST_ID
        , LEFT(CASTID, 1) as Strand
        , RIGHT(CASTID, 2) as Sweep
        , SEQ.TAPDATE as TAP_DATE
        , DEFECT_CLASS as CLASS
        , DEFECT_CODE AS CODE
        , CONCAT(HEAT, LEFT(CASTID, 1), RIGHT(CASTID, 2)) as Bloom_ID
    FROM SQIS11_LINK..QA.C_INSPEC inspec, SQIS11_LINK..QA.C_SEQ seq
    WHERE inspec.HEAT_NUMBER = SEQ.HEAT
    AND inspec.RAIL_MILL_LETTER = SEQ.RMLETTER
    AND inspec.RAIL_SEQ_NO = SEQ.RMSEQ
    AND inspec.DEFECT_CODE IN (16, 20, 25, 33, 52, 55, 69, 70)
    AND HEAT > 576476
    UNION
    SELECT
        CONCAT(
            SEQ.HEAT
            , SEQ.RMSEQ
            , SEQ.RMLETTER
        ) as RAIL
        , HEAT
        , CONCAT( SEQ.RMSEQ, SEQ.RMLETTER) AS RAIL_ID
        , CASTID as CAST_ID
        , LEFT(CASTID, 1) as Strand
        , RIGHT(CASTID, 2) as Sweep
        , SEQ.TAPDATE as TAP_DATE
        , DEFECT_CLASS as CLASS
        , DEFECT_CODE AS CODE
        , CONCAT(HEAT, LEFT(CASTID, 1), RIGHT(CASTID, 2)) as Bloom_ID
    FROM SQIS11_LINK..QA.REWORK rework, SQIS11_LINK..QA.C_SEQ seq
    WHERE rework.HEAT_NUMBER = SEQ.HEAT
    AND rework.RAIL_MILL_LETTER = SEQ.RMLETTER
    AND rework.RAIL_SEQ_NO = SEQ.RMSEQ
    AND rework.DEFECT_CODE IN (16, 20, 25, 33, 52, 55, 69, 70)
    AND HEAT > 576476
    order by seq.TAPDATE desc, rail_ID asc
    ;
    '''
# Bryce and I jointly defined an SQL Query to retrieve data specific to each rail tested including
# the batch (heat) number, the casting position (Strand and Sweep),
# the defect class and code, and a unique Bloom ID identifier to remove duplicates.

In [5]:
'''
Author: Bryce Lakamp, Senior Process Engineer, Evraz
Date: January 2023
bryce.lakamp@evrazna.com
'''
if __name__ == "__main__":
    from IPython.display import display_html
    conn_1 = rm_l2_mirror_conn_str()

    sql_1 = '''
        SELECT HEAT, count([HEAT]) charged, count(NDT_FLAG) NDT
        FROM SQIS11_LINK..QA.C_SEQ
        group by HEAT;
        '''
# Bryce and I jointly defined an SQL query to retrieve total number of rails charged into the furnace per heat
# and the total number of rails that were run through the Non-Destructive Test Suite to look for the defect of interest.


In [6]:
if __name__ == "__main__":
    from IPython.display import display_html
    conn_2 = st_l2_mirror_conn_str()
    sql_2 = '''
    SELECT
        HeatNumber AS HEAT
       , HeatGrade
       , CastSequence
       , CasterHeatSequence
       , CasterGradeCode
       , MONTH(CastStartTime) as [Month]
       , YEAR(CastStartTime) as [Year]
       , CastStartTime
       , CastEndTime
       , CasterHeatTime
       , LadleWeightTons
       , CasterSteelWeightTons
       , CasterSupervisorName
       , CasterMoldOperatorNames
       , TundishWeightAvg
       , TundishPreHeatTime
       , TundishPreHeatTemperature
       , TundishTemperature
       , TundishCoverMaterial
       , LEFT(TundishIdentifier,3) AS Tundish
       , CasterNumberOfShims
       , CasterAvgSubmergence
       , CasterSulfur
       , LiquidousTemperature
       , (CasterLadleTemperature - LiquidousTemperature) as SuperHeat
       , CasterLadleOpenFLP
       , CasterLadleNumber
       , CasterLadleShroudSupplier
       , CasterLadleTemperature
       , CasterVortexDetectedTimeStamp
       , CasterSlagDetectedTimeStamp
       , ((Strand1CastSpeed+Strand2CastSpeed+Strand3CastSpeed+Strand4CastSpeed+Strand5CastSpeed+Strand6CastSpeed)/6) AS CastSpeed
   FROM HeatSingleRecord
   WHERE BilletSize > 11
   AND HeatNumber > 576476
   ;
    '''
# Independently developed SQL Query based on previous two to collect
# casting variables of interest related to each heat, to use in visualizations.
# Limited each query by product size and batch number to reduce the total size.

In [7]:
df = pd.read_sql(sql, con)
df.drop_duplicates(['RAIL'], inplace=True)
df['CLASS']=df['CLASS'].replace('W','CUT')
df['CLASS']=df['CLASS'].replace('S','SCRAP')
df['CLASS']=df['CLASS'].replace('Z','INDUSTRIAL RAIL')
df['HEAT']=df['HEAT'].astype('int64')
df['Strand']=df['Strand'].astype('int64')
df['Sweep']=df['Sweep'].astype('int64')
df['CODE']=df['CODE'].astype('int64')
df['Bloom_ID']=df['Bloom_ID'].astype('int64')

# reading the data from the first query into a df
# changing some data types from original to ensure functionality in pandas

In [8]:
df_1 = pd.read_sql(sql_1, conn_1)
df_1['HEAT']=df_1['HEAT'].astype('int64')

# reading the data from the second query into a df
# changing some data types from original to ensure functionality in pandas

In [9]:
df_2 = pd.read_sql(sql_2, conn_2)

# reading the data from the third query into a df

In [80]:
df_Variables = pd.merge(df_1, df_2, on='HEAT')

# combining two of the data frame reads to ensure variables are all matched against the same batch product

,HEAT,charged,NDT,HeatGrade,CastSequence,CasterHeatSequence,CasterGradeCode,Month,Year,CastStartTime,CastEndTime,CasterHeatTime,LadleWeightTons,CasterSteelWeightTons,CasterSupervisorName,CasterMoldOperatorNames,TundishWeightAvg,TundishPreHeatTime,TundishPreHeatTemperature,TundishTemperature,TundishCoverMaterial,Tundish,CasterNumberOfShims,CasterAvgSubmergence,CasterSulfur,LiquidousTemperature,SuperHeat,CasterLadleOpenFLP,CasterLadleNumber,CasterLadleShroudSupplier,CasterLadleTemperature,CasterVortexDetectedTimeStamp,CasterSlagDetectedTimeStamp,CastSpeed
4435,576477,56,54,TC8086,5873,20,220,1,2015,2015-01-01 10:54:04.000,2015-01-01 11:44:28.000,50.400002,189.380005,126.430000,"Keys, Craig","O''Neil, Ed / Varner, Jeffrey",0.000000,3.3,2120.0,2701.496094,T715G2,14E,4.0,4.0,0.013,2651.0,126.0,F,29,EIC,2777.0,2015-01-01 11:36:51,2015-01-01 11:38:25,25.109015
17505,576478,42,41,TC8086,5874,1,220,1,2015,2015-01-01 17:19:13.000,2015-01-01 23:51:13.000,392.000000,195.029999,132.380005,"Keys, Craig","O''Neil, Ed / Varner, Jeffrey",3.188984,3.3,2120.0,2727.142090,T715G2,17W,4.0,4.0,0.010,2652.0,176.0,L6,23,EIC,2828.0,NaT,NaT,2.834500
8720,576479,64,63,TC8086,5874,2,220,1,2015,2015-01-01 18:17:34.000,2015-01-01 18:59:16.000,41.700001,194.270004,130.320007,"Keys, Craig","O''Neil, Ed / Varner, Jeffrey",0.000000,3.3,2120.0,2736.305908,T715G2,17W,3.0,4.5,0.011,2649.0,124.0,L4,27,EIC,2773.0,2015-01-01 18:56:05,NaT,28.742674
21803,576480,60,54,TC8086,5874,3,220,1,2015,2015-01-01 18:59:33.000,2015-01-01 19:51:57.000,52.400002,195.380005,130.729996,"Kennedy, Eric","Villegas, Don / Padilla, Leopoldo",1.122345,3.3,2120.0,2720.975098,T715G2,17W,2.0,5.0,0.017,2648.0,113.0,F,31,EIC,2761.0,2015-01-01 19:47:56,2015-01-01 19:48:58,22.695810
30549,576481,52,52,TC8086,5874,4,220,1,2015,2015-01-01 19:52:14.000,2015-01-01 20:41:32.000,49.299999,185.740005,120.839996,"Kennedy, Eric","Villegas, Don / Padilla, Leopoldo",0.000000,3.3,2120.0,2703.590088,T715G2,17W,1.0,5.5,0.016,2649.0,127.0,L3,30,EIC,2776.0,2015-01-01 20:38:52,2015-01-01 20:40:16,22.447250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30514,635268,72,53,TC8086,7431,15,270,2,2023,2023-02-17 06:05:43.000,2023-02-17 07:10:01.000,64.300003,186.899994,126.900002,"Cesario, Jason","Nelson, David / Lawrence, Jeremy",0.000000,4.7,2234.0,2706.472900,715,19E,NaN,NaN,0.016,2649.0,102.0,F,33,VES,2751.0,2023-02-17 07:08:21,2023-02-17 07:07:58,22.254042
26219,635269,64,0,TC8086,7431,16,270,2,2023,2023-02-17 07:10:17.520,2023-02-17 08:05:05.520,54.799999,177.649994,117.650002,"Cesario, Jason","Nelson, David / Lawrence, Jeremy",0.000000,4.7,2234.0,2728.689941,715,19E,NaN,NaN,0.014,2649.0,135.0,F,22,VES,2784.0,NaT,NaT,24.118597
30513,635270,68,0,TC8086,7431,17,270,2,2023,2023-02-17 08:05:26.000,2023-02-17 08:58:20.000,52.900002,186.600006,126.599998,"Cesario, Jason","Nelson, David / Lawrence, Jeremy",0.000000,4.7,2234.0,2725.313965,715,19E,NaN,NaN,0.014,2650.0,113.0,F,25,VES,2763.0,NaT,2023-02-17 08:56:29,26.080017
21771,635271,66,4,TC8086,7431,18,270,2,2023,2023-02-17 08:58:38.000,2023-02-17 09:46:44.000,48.099998,180.699997,120.699997,"Cesario, Jason","Nelson, David / Lawrence, Jeremy",0.000000,4.7,2234.0,2718.358887,715,19E,NaN,NaN,0.014,2650.0,98.0,F,36,VES,2748.0,2023-02-17 09:44:20,2023-02-17 09:44:22,27.824318


In this project, my knowledge of the process let me decide which outliers were simple impossible. For example, reported temperatures over 20,000 F are physically impossible in the process. I chose to replace these outliers with NaN because it let me keep the rest of the information in the row that was reasonable, without introducing estimations into the data. This only works because I will be building comparison visuals; machine learning would require that I replace the outliers, with usable data, likely with averages for the column determined by the product type.

In [81]:
df_Variables["SuperHeat"].mask((df_Variables["SuperHeat"] < 0) | (df_Variables["SuperHeat"] > 200), np.nan, inplace=True)
df_Variables["CasterLadleTemperature"].mask((df_Variables["CasterLadleTemperature"] <= 1000) | 
                                            (df_Variables["CasterLadleTemperature"] > 3000), np.nan, inplace=True)
df_Variables["CastSpeed"].mask((df_Variables["CastSpeed"] < 10), np.nan, inplace=True)
df_Variables["LiquidousTemperature"].mask((df_Variables["LiquidousTemperature"] <= 1000) | 
                                          (df_Variables["LiquidousTemperature"] > 3000), np.nan, inplace=True)
df_Variables["CasterAvgSubmergence"].mask((df_Variables["CasterAvgSubmergence"] > 15), np.nan, inplace=True)
df_Variables["TundishTemperature"].mask((df_Variables["TundishTemperature"] <= 1000) | 
                                        (df_Variables["TundishTemperature"] > 3000), np.nan, inplace=True)

# this series is removing outliers and replacing with NaN. 

For ease of use in a visualization program like PowerBI, I wanted all of the data condensed into a single dataframe, all collected and referenced by the Heat number, which is essentially my product ID.

In [87]:
df_Inclusion_Count = df.groupby(by="HEAT")["RAIL"].count()
# grouping the df that contains the defect data by batch of product made

df_Inclusions = pd.merge(df_Variables, df_Inclusion_Count, on='HEAT',how='outer')
# combining df's so that variables and defect counts are in the same frame

df_Inclusions.sort_values(by=['HEAT'],inplace=True)
# sorting the df so it is in timeline order of when the products were made.

In [89]:
with pd.ExcelWriter(r'File Path to Excel for PowerBI.xlsx',
                    mode='a', if_sheet_exists='replace') as writer:
    df_Inclusions.to_excel(writer, sheet_name='Data')
    
# This exports the final df into an excel sheet that is accessed by PowerBI. The file path in this code is a dummy path.